<a href="https://colab.research.google.com/github/alangan17/smazc02/blob/main/SMAZC_02_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Module 2 Homework

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [Colab](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/Module2_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.


## Import

In [1]:
!pip install yfinance

In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

## Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happened on Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).

Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).

You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 25 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)

In [53]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [57]:
ipo_dfs[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     325 non-null    object
 1   Symbol          325 non-null    object
 2   Company Name    325 non-null    object
 3   Price Range     325 non-null    object
 4   Shares Offered  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [58]:
df = ipo_dfs[0]
df

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
...,...,...,...,...,...
320,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
321,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
322,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
323,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [59]:
# convert to datetime
df['Filing Date'] = pd.to_datetime(df['Filing Date'], format="%b %d, %Y")

In [62]:
# Filter only year 2023 data
df = df[df['Filing Date'].dt.year == 2023]

# Filter only Friday
df = df[df['Filing Date'].dt.dayofweek == 4]

In [64]:
df.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000
51,2023-12-29,EPSM,Epsium Enterprise Limited,-,-
54,2023-12-22,LZMH,LZ Technology Holdings Limited,-,-
55,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,-,-
60,2023-12-15,GIT,Going International Holding Company Limited,-,-


In [65]:
# it has some missing values --> use defensive errors='coerce' (if don't have time to crack into the data errors)
#     : pd.to_numeric() function call, which will convert problematic values to NaN.
#     otherwise you'll get a ValueError: Unable to parse string "-" at position 9
df['Shares Offered'] = pd.to_numeric(df['Shares Offered'].str.replace('$', ''), errors='coerce')
# not sure why, but need to call it again to transform 'object' to 'float64'
df['Shares Offered'] = pd.to_numeric(df['Shares Offered'])


In [66]:
# Convert `Price Range` to `Avg_price`
# First, let's convert any "-" in the "Price Range" column to NaN
df['Price Range'] = df['Price Range'].replace('-', pd.NA)

# Then, split the "Price Range" column into two columns using "-" as a delimiter
price_range_split = df['Price Range'].str.split(' - ', expand=True)

# Assign the first and second parts to new columns
df['Lower Price'] = price_range_split[0].str.replace('$', '')
df['Upper Price'] = price_range_split[1].str.replace('$', '')

# Convert the new columns to numeric data type
df['Lower Price'] = pd.to_numeric(df['Lower Price'], errors='coerce')
df['Upper Price'] = pd.to_numeric(df['Upper Price'], errors='coerce')
df['Avg_price'] = (df['Lower Price'].fillna(df['Upper Price']) + df['Upper Price'].fillna(df['Lower Price'])) / 2

In [68]:
# Calculate `Shares_offered_value`
df['Shares_offered_value'] = df['Avg_price'] * df['Shares Offered']

In [77]:
df = df[df['Shares Offered'].notnull()]
df

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Lower Price,Upper Price,Avg_price,Shares_offered_value
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000.0,3.5,4.5,4.00,4800000.0
62,2023-12-08,ENGS,Energys Group Limited,$4.00 - $6.00,2000000.0,4.0,6.0,5.00,10000000.0
63,2023-12-08,LNKS,Linkers Industries Limited,$4.00 - $6.00,2200000.0,4.0,6.0,5.00,11000000.0
82,2023-10-27,RAY,Raytech Holding Limited,$4.00 - $5.00,1500000.0,4.0,5.0,4.50,6750000.0
89,2023-10-13,ORIS,Oriental Rise Holdings Limited,$4.00,2000000.0,4.0,NaN,4.00,8000000.0
92,2023-10-06,QMMM,QMMM Holdings Limited,$4.00,2125000.0,4.0,NaN,4.00,8500000.0
96,2023-09-29,KAPA,"Kairos Pharma, Ltd.",$4.00,1550000.0,4.0,NaN,4.00,6200000.0
97,2023-09-29,VAPA,Valens Pay Global Limited,$5.00 - $6.00,1000000.0,5.0,6.0,5.50,5500000.0
103,2023-09-15,ACSB,Acesis Holdings Corporation,$4.00 - $6.00,1300000.0,4.0,6.0,5.00,6500000.0
121,2023-07-07,AZI,Autozi Internet Technology (Global) Ltd.,$4.00 - $5.00,1250000.0,4.0,5.0,4.50,5625000.0


In [80]:
# What's the total sum ($m) of 2023 filings that happenned of Fridays?
round(df['Shares_offered_value'].sum() / 1000000, 0)

286.0

## Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/).
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this [link](https://stockanalysis.com/actions/changes/).
Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.


Additional:
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [87]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

# Step 1: Retrieve IPO data from URLs
url_2023 = "https://stockanalysis.com/ipos/2023/"
url_2024 = "https://stockanalysis.com/ipos/2024/"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response1 = requests.get(url_2023, headers=headers)
response2 = requests.get(url_2024, headers=headers)

ipos_2023 = pd.read_html(response1.text)[0]
ipos_2024 = pd.read_html(response2.text)[0]

# Step 2: Filter IPOs before March 1, 2024 and remove 'RYZB'
ipos_2023['IPO Date'] = pd.to_datetime(ipos_2023['IPO Date'], format="%b %d, %Y")
ipos_2024['IPO Date'] = pd.to_datetime(ipos_2024['IPO Date'], format="%b %d, %Y")

ipos_2023_filtered = ipos_2023[ipos_2023['IPO Date'] < datetime(2024, 3, 1)]
ipos_2024_filtered = ipos_2024[ipos_2024['IPO Date'] < datetime(2024, 3, 1)]

ipos_filtered = pd.concat([ipos_2023_filtered, ipos_2024_filtered], ignore_index=True)
ipos_filtered = ipos_filtered[ipos_filtered['Symbol'] != 'RYZB']

ipos_filtered


,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.04,0.50%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.90,-26.25%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.43,-20.32%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.61,-84.73%
...,...,...,...,...,...,...
180,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$16.57,1.22%
181,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$2.70,-58.83%
182,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$2.90,-27.63%
183,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$29.00,39.10%


In [106]:
# Step 3: Retrieve OHLCV data for each stock
def get_data(symbol, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        return data
    except:
        return None

end_date = datetime(2024, 3, 1)
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

ohlcvs = {}
for symbol in ipos_filtered['Symbol']:
    data = get_data(symbol, start_date, end_date)
    if data is not None:
        data.index = pd.to_datetime(data.index)  # Convert index to date datatype
        ohlcvs[symbol] = data


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [103]:
ohlcvs['IROH'].index.max()

Timestamp('2024-02-29 00:00:00')

In [99]:
ipos_filtered[ipos_filtered['Symbol']=='IROH']

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.04,0.50%


In [108]:
def get_growth_df(df:pd.DataFrame, prefix:str)->pd.DataFrame:
  for i in [1,3,7,30,90,365]:
    df['growth_'+prefix+'_'+str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)
    GROWTH_KEYS = [k for k in df.keys() if k.startswith('growth')]
  return df[GROWTH_KEYS]

In [136]:
growth_df = {}
for k, v in ohlcvs.items():
    temp_growth_df = get_growth_df(df=v, prefix="")
    growth_df[k] = temp_growth_df.tail(1)[]

In [137]:
growth_df['AHR']

,growth_AHR_1d,growth_AHR_3d,growth_AHR_7d,growth_AHR_30d,growth_AHR_90d,growth_AHR_365d,growth__1d,growth__3d,growth__7d,growth__30d,growth__90d,growth__365d
Date,,,,,,,,,,,,
2024-02-29,1.002199,1.03092,1.019389,NaN,NaN,NaN,1.002199,1.03092,1.019389,NaN,NaN,NaN


In [123]:
merged_growth_df = pd.concat(growth_df.values(), ignore_index=False)

In [122]:
merged_growth_df

,growth_IROH_1d,growth_IROH_3d,growth_IROH_7d,growth_IROH_30d,growth_IROH_90d,growth_IROH_365d,growth_LGCB_1d,growth_LGCB_3d,growth_LGCB_7d,growth_LGCB_30d,...,growth_SDHC_7d,growth_SDHC_30d,growth_SDHC_90d,growth_SDHC_365d,growth_ROMA_1d,growth_ROMA_3d,growth_ROMA_7d,growth_ROMA_30d,growth_ROMA_90d,growth_ROMA_365d
0,1.0,0.996008,0.996008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,0.975369,0.99,1.076087,1.202186,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.031843,1.204678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
# Step 4: Calculate growth for each holding period (X days)
days = np.arange(1, 31)
growth_df = pd.DataFrame(index=ipos_filtered.index, columns=[f'growth_future_{d}d' for d in days])

for i, row in ipos_filtered.iterrows():
    symbol = row['Symbol']
    ipo_date = row['IPO Date']
    print(f'{symbol=}')
    close_price = ohlcvs[symbol].loc[ipo_date]['Adj Close']

    for d in days:
        sell_date = ipo_date + timedelta(days=d)
        if sell_date in ohlcvs[symbol].index:
            sell_price = ohlcvs[symbol].loc[sell_date]['Adj Close']
            growth_df.at[i, f'growth_future_{d}d'] = (sell_price - close_price) / close_price
        else:
            growth_df.at[i, f'growth_future_{d}d'] = np.nan

# Step 5: Find optimal X by maximizing 75th percentile growth
percentile_75th = growth_df.quantile(0.75)
optimal_X = percentile_75th.idxmax().split('_')[-1][:-1]

# Step 6: Analyze recommendation
mean_growth = growth_df.mean()
median_growth = growth_df.median()

print("Optimal X:", optimal_X)
print("\nMean Growth:\n", mean_growth)
print("\nMedian Growth:\n", median_growth)

symbol='IROH'


KeyError: Timestamp('2023-12-27 00:00:00')

## Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks
for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate  `growth_7d` for every stock and every day.
Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

### Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years?
One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit.
You need to sum these results over all trades (460 times in 10 years).

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?


### [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?
